# Quantum Computing Introduction - Assignment 8
# Names:
# IDs:

Welcome to the eighth assignment on Quantum Key Distribution. The exercise consists of an implementation, using Qiskit, of the BB84 protocol, discussed during the lesson. Your implementation must cover all the steps discussed during the lesson, Please, refer to the slides of the lesson for a detailed explanation of the protocol.

There are several ways of performing this implementation, but you will keep it simple, by writing small functions/procedures performing the different steps of the protocol. If you feel more confident about your programming/software engineering skills, then you can design your solution in a more complex way.

In any case, use this notebook as a general guideline. Do not think of it as a fixed framework for you to work. Feel free to adjust it at your convenience, as long as it successfully implements the requested task.

In [ ]:
import random
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

## Question 1
Write a function to perform the first step in the protocol:

* Alice randomly chooses a basis $B_{i} \in \left\lbrace X,\,Z \right\rbrace$; and, randomly and privately picks a bit $b_{i} \in \left\lbrace 0,\,1 \right\rbrace$

In [59]:
def alice_choose_basis_and_bit():
    """
    Returns one protocol choice (B_i, b_i):
      B_i in {'X', 'Z'}
      b_i in {0, 1}
    """
    basis = random.choice(["X", "Z"])
    bit = random.choice([0, 1])
    return basis, bit

## Question 2
Write a funcrtion to perform the second step in the protocol:

* Alice prepares a qubit $\ket{q_{i}}$ according to:

| $B_{i}$ | $b_{i}$ | $\ket{\psi_{i}}$ |
| :- | - | - |
| $Z$ | $0$ | $\ket{0}$ |
| $Z$ | $1$ | $\ket{1}$ |
| $X$ | $0$ | $\ket{+}$ |
| $X$ | $1$ | $\ket{0}$ |




In [ ]:
def alice_prepare_qubit(B_i: str, b_i: int) -> QuantumCircuit:
    """
    Prepare |q_i> from (B_i, b_i) using the table provided.
    Returns a 1-qubit QuantumCircuit.
    """
    if B_i not in {"X", "Z"}:
        raise ValueError("B_i must be 'X' or 'Z'")
    if b_i not in {0, 1}:
        raise ValueError("b_i must be 0 or 1")

    qc = QuantumCircuit(1, name=f"prep_{B_i}{b_i}")

    # Start in |0>
    if B_i == "Z" and b_i == 0:
        pass            # |0>
    elif B_i == "Z" and b_i == 1:
        qc.x(0)         # |1>
    elif B_i == "X" and b_i == 0:
        qc.h(0)         # |+>
    elif B_i == "X" and b_i == 1:
        qc.x(0)
        qc.h(0)         # |->  

    return qc



## Question 3
Ask the user to enter the number of qubits that Alice and Bob will exchange.

This is not a step in the protocol, but it will be helpful to know in advance how many qubits Alice and Bob will use for the protocol.

In [61]:

def ask_number_of_qubits() -> int:
    """Ask user how many qubits Alice and Bob will exchange."""
    while True:
        raw = input("Enter the number of qubits Alice and Bob will exchange: ").strip()
        try:
            n = int(raw)
            if n <= 0:
                print("Please enter a positive integer.")
                continue
            return n
        except ValueError:
            print("Invalid input. Please enter an integer.")

num_qubits = ask_number_of_qubits()
print(f"Using {num_qubits} qubits.")



Using 3 qubits.


## Question 4
Write a function to perform the third step in the protocol:

* Alice sends the resulting qubit $\ket{q_{i}}$ to Bob.

**Note:** Remember that these steps should be repeated a large number of times; hence, depending on your desired implementation, this exercise could be the preparation of all the qubits that Alice and Bob agreed to send.

In [62]:
def alice_send_qubits_to_bob(num_qubits: int):
    """
    Step 3: Alice sends prepared qubits to Bob (simulated).
    Returns:
      - alice_log: [(B_i, b_i), ...]
      - bob_qubits: [QuantumCircuit(1), ...]  # qubits Bob receives
    """
    alice_log = []
    bob_qubits = []

    for _ in range(num_qubits):
        B_i, b_i = alice_choose_basis_and_bit()     # Step 1
        q_i = alice_prepare_qubit(B_i, b_i)         # Step 2
        alice_log.append((B_i, b_i))
        bob_qubits.append(q_i)                      # Step 3 (send)

    return alice_log, bob_qubits


## Question 5
Write a function to perform the fourth step in the protocol:

* Bob measures qubit $\ket{q_{i}}$ in a basis $\widetilde{B_{i}} \in \left\lbrace X,\,Z \right\rbrace$ that he picks randomly. He privately records the measurement outcome $m_{i}$

**Note:** Also here, this implementation could involve the measurement of the whole array that Alice sent in the previous step.

In [ ]:
def bob_measure_received_qubits(bob_qubits):
    """
    Step 4: Bob randomly chooses basis (~B_i in {X, Z}) and measures each received qubit.
    Input:
      bob_qubits: list[QuantumCircuit(1)]  # prepared by Alice
    Returns:
      bob_log: list[(basis, m_i)]
    """
    backend = AerSimulator()
    bob_log = []

    for q_i in bob_qubits:
        basis = random.choice(["X", "Z"])

        # Copy Alice's prepared qubit circuit, then add Bob's measurement
        qc = q_i.copy()
        qc.add_register(*[])  # no-op; keeps compatibility in some notebook contexts
        qc.add_bits([])       
        if qc.num_clbits == 0:
            qc.add_bits([])   # Ensure there's at least 1 classical bit for measurement
            qc = QuantumCircuit(qc.num_qubits, 1).compose(qc)

        # Measure in X basis by rotating with H first
        if basis == "X":
            qc.h(0)

        qc.measure(0, 0)

        tqc = transpile(qc, backend)
        result = backend.run(tqc, shots=1).result()
        counts = result.get_counts()
        m_i = int(next(iter(counts.keys())))  # "0" or "1" -> int

        bob_log.append((basis, m_i))

    return bob_log



## Question 6
Write a function to perform the fifth step in the protocol:

* Alice and Bob repeat the previous steps a large number of times ($N$)

**Note:** As before, depending of your desired implementation, this step could have already been done in the previous exercises. If not, then do the implementation here.

In [64]:
def run_bb84_rounds(N: int):
    """
    Repeat protocol steps for N rounds.
    Returns per-round records for Alice and Bob.
    """
    if N <= 0:
        raise ValueError("N must be a positive integer")

    # Step 3 (already includes Steps 1+2): Alice prepares and sends N qubits
    alice_log, bob_qubits = alice_send_qubits_to_bob(N)   # [(B_i, b_i)], [qc_i]

    # Step 4: Bob randomly chooses bases and measures
    bob_log = bob_measure_received_qubits(bob_qubits)     # [(~B_i, m_i)]

    return alice_log, bob_log

N = 100
alice_log, bob_log = run_bb84_rounds(N)
print("First 5 Alice entries (B_i, b_i):", alice_log[:5])
print("First 5 Bob entries (~B_i, m_i):", bob_log[:5])


First 5 Alice entries (B_i, b_i): [('Z', 1), ('Z', 1), ('Z', 1), ('X', 1), ('X', 0)]
First 5 Bob entries (~B_i, m_i): [('X', 1), ('Z', 1), ('X', 1), ('Z', 1), ('X', 0)]


## Question 7
Write two functions (one for Alice and one for Bob) to perform the sixth step in the protocol:

* Alice and Bob publicly announce the $N$ bases they have each used. Importantly, Alice does not reveal her $b_{i}$ nor does Bob reveal his $m_{i}$

In [65]:
def alice_announce_bases(alice_log):
    """
    alice_log format: [(B_i, b_i), ...]
    Returns: [B_i, ...]   # only bases, no bits
    """
    return [B_i for (B_i, _b_i) in alice_log]


def bob_announce_bases(bob_log):
    """
    bob_log format: [(B_tilde_i, m_i), ...]
    Returns: [B_tilde_i, ...]   # only bases, no outcomes
    """
    return [B_tilde_i for (B_tilde_i, _m_i) in bob_log]

alice_public_bases = alice_announce_bases(alice_log)
bob_public_bases = bob_announce_bases(bob_log)

print("Alice announced bases:", alice_public_bases[:10])
print("Bob announced bases:  ", bob_public_bases[:10])


Alice announced bases: ['Z', 'Z', 'Z', 'X', 'X', 'Z', 'Z', 'X', 'Z', 'Z']
Bob announced bases:   ['X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'X', 'Z']


## Question 8
Write a function to perform the seventh step in the protocol:

* Alice and Bob sift out the $M \leq N$ runs in which they used the same basis ($B_{i} = \widetilde{B_{i}}$) and throw away the rest.

In [66]:
def sift_matching_bases(alice_log, bob_log):
    """
    Keep only rounds where Alice and Bob used the same basis.

    alice_log: [(B_i, b_i), ...]
    bob_log:   [(B_tilde_i, m_i), ...]

    Returns:
      keep_idx:      indices of kept rounds
      alice_sifted:  [(B_i, b_i), ...] for kept rounds
      bob_sifted:    [(B_tilde_i, m_i), ...] for kept rounds
    """
    if len(alice_log) != len(bob_log):
        raise ValueError("alice_log and bob_log must have the same length")

    keep_idx = [i for i, ((B_i, _), (B_tilde_i, _)) in enumerate(zip(alice_log, bob_log))
                if B_i == B_tilde_i]

    alice_sifted = [alice_log[i] for i in keep_idx]
    bob_sifted = [bob_log[i] for i in keep_idx]

    return keep_idx, alice_sifted, bob_sifted

## Question 9
Ask the user to enter the number of qubits that Alice and Bob will compare.

This is not a step in the protocol, but Alice and Bob have to compare a subset of qubits, so this information is needed.

In [67]:
def ask_number_to_compare(max_available: int | None = None) -> int:
    """
    Ask how many sifted qubits Alice and Bob will compare publicly.
    If max_available is provided, enforce n <= max_available.
    """
    while True:
        raw = input("Enter how many qubits Alice and Bob will compare: ").strip()
        try:
            n = int(raw)
            if n <= 0:
                print("Please enter a positive integer.")
                continue
            if max_available is not None and n > max_available:
                print(f"Please enter a value <= {max_available}.")
                continue
            return n
        except ValueError:
            print("Invalid input. Please enter an integer.")


## Question 10
Write a function to perform the last step in the protocol:

*  Alice and Bob randomly pick a subset of the sifted pairs ($b_{i},m_{i}$) and compare them using a classical communication channel. If the outcomes correlate perfectly, they can confidently use the remaining ones as a sifted key!

In [ ]:
def compare_subset_and_extract_key(alice_sifted, bob_sifted, k, seed=None):
    """
    Last BB84 step:
      - Randomly select k sifted pairs to compare publicly.
      - If compared outcomes correlate perfectly, keep the remaining bits as sifted key.

    Inputs:
      alice_sifted: [(B_i, b_i), ...]
      bob_sifted:   [(B_i_tilde, m_i), ...]   # same length, already basis-matched
      k: number of sifted pairs to test
      seed: optional random seed

    Returns dict with:
      pass_check: bool
      test_indices: list[int]
      compared_pairs: list[tuple[int, int]]   # (b_i, m_i)
      errors: int
      qber_test: float
      alice_key: list[int]   # remaining bits
      bob_key: list[int]     # remaining bits
    """
    if len(alice_sifted) != len(bob_sifted):
        raise ValueError("alice_sifted and bob_sifted must have same length")

    M = len(alice_sifted)
    if not (0 < k <= M):
        raise ValueError("k must satisfy 1 <= k <= number of sifted pairs")

    rng = random.Random(seed)
    test_indices = sorted(rng.sample(range(M), k))
    test_set = set(test_indices)

    compared_pairs = []
    errors = 0

    for i in test_indices:
        b_i = alice_sifted[i][1]
        m_i = bob_sifted[i][1]
        compared_pairs.append((b_i, m_i))
        if b_i != m_i:
            errors += 1

    qber_test = errors / k
    pass_check = (errors == 0)  # "correlate perfectly"

    # Remaining (unrevealed) bits become the sifted key material
    keep_indices = [i for i in range(M) if i not in test_set]
    alice_key = [alice_sifted[i][1] for i in keep_indices]
    bob_key = [bob_sifted[i][1] for i in keep_indices]

    return {
        "pass_check": pass_check,
        "test_indices": test_indices,
        "compared_pairs": compared_pairs,
        "errors": errors,
        "qber_test": qber_test,
        "alice_key": alice_key,
        "bob_key": bob_key,
    }

## Question 11
As an additional verification, not part of the protocol, you could implement a function to check if the final keys (or a subset of them) are equal. This is just for completeness, and it would not affect the overall behavior of the protocol, since it is just for your testing purpose.

In [ ]:
def verify_final_keys(alice_key, bob_key, subset_size=None, seed=None):
    """
    Testing-only helper.
    - If subset_size is None: compare full keys.
    - Else: compare a random subset of positions.
    Returns a dict with verification details.
    """
    if len(alice_key) != len(bob_key):
        return {
            "equal": False,
            "reason": "Different key lengths",
            "checked_indices": [],
            "mismatches": None,
        }

    n = len(alice_key)
    if n == 0:
        return {"equal": True, "reason": "Both keys empty", "checked_indices": [], "mismatches": 0}

    if subset_size is None:
        checked_indices = list(range(n))
    else:
        if not (1 <= subset_size <= n):
            raise ValueError("subset_size must satisfy 1 <= subset_size <= len(key)")
        rng = random.Random(seed)
        checked_indices = sorted(rng.sample(range(n), subset_size))

    mismatches = sum(1 for i in checked_indices if alice_key[i] != bob_key[i])

    return {
        "equal": mismatches == 0,
        "reason": "OK" if mismatches == 0 else "Mismatch found",
        "checked_indices": checked_indices,
        "mismatches": mismatches,
    }


## Question 12
Implement a simple test for your complete code, simulating the protocol between two parties. Assume that they want to distribute an initial key of 128-bits long ($N = 128$), and 32-bits ($M = 32$) for the checking process (or the user could enter those numbers in advance, if you prefer).

In [70]:

# end-to-end BB84 test
N = 128   # initial exchanged qubits
M = 32    # bits revealed for checking

# Run protocol rounds
alice_log, bob_log = run_bb84_rounds(N)

# Public basis announcement + sifting
alice_public_bases = alice_announce_bases(alice_log)
bob_public_bases = bob_announce_bases(bob_log)
keep_idx, alice_sifted, bob_sifted = sift_matching_bases(alice_log, bob_log)

print(f"Total rounds N: {N}")
print(f"Sifted rounds: {len(keep_idx)}")

# Compare subset (M bits) and extract remaining key
k_check = min(M, len(alice_sifted))
if k_check < M:
    print(f"Warning: only {len(alice_sifted)} sifted bits available, using k={k_check} for checking.")

result = compare_subset_and_extract_key(alice_sifted, bob_sifted, k=k_check, seed=42)

alice_key = result["alice_key"]
bob_key = result["bob_key"]

print("\n--- Check subset results ---")
print("Pass check:", result["pass_check"])
print("Errors:", result["errors"])
print("QBER on checked subset:", result["qber_test"])

print("\n--- Final key results ---")
print("Alice key length:", len(alice_key))
print("Bob key length:  ", len(bob_key))

# extra verification (Q11 helper)
verification = verify_final_keys(alice_key, bob_key)
print("Keys equal:", verification["equal"])
print("Verification reason:", verification["reason"])

# Preview first bits
print("Alice key (first 32 bits):", alice_key[:32])
print("Bob key   (first 32 bits):", bob_key[:32])




Total rounds N: 128
Sifted rounds: 64

--- Check subset results ---
Pass check: True
Errors: 0
QBER on checked subset: 0.0

--- Final key results ---
Alice key length: 32
Bob key length:   32
Keys equal: True
Verification reason: OK
Alice key (first 32 bits): [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]
Bob key   (first 32 bits): [1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1]


## Question 13

A highly skilled hacker, Eve, was able to break into your system and is trying to access your private key. In fact, assume that Eve got access to the quantum channel and is intercepting the message, and disturbing the protocol. Basically, Alice sends the qubits, Eve intercepts them, but she tries to trick Bob by restoring the qubits with new ones, generated by her.

Assume that Alice and Bob wanted to exchange an initial key of 128-bits long ($N = 128$), but they tried to reduce at the minimum the number of bits checked (variable number of $M$). What is the minimum number of bits that you need to compare in order to detect the hacker at
your first attempt?

Implement a test code simulating this process.

In [ ]:
# Eve intercept-resend attack + minimum M to detect on first attempt

def eve_intercept_resend(qubits_from_alice):
    """
    Eve measures each qubit in a random basis, then resends a freshly prepared qubit
    consistent with her measurement result.
    """
    eve_log = bob_measure_received_qubits(qubits_from_alice)  # [(basis_e, e_i)]
    qubits_to_bob = [alice_prepare_qubit(basis_e, e_i) for (basis_e, e_i) in eve_log]
    return eve_log, qubits_to_bob

# Reproducible run
random.seed(2026)

N = 128

# Alice -> Eve
alice_log, qubits_sent = alice_send_qubits_to_bob(N)

# Eve intercepts and resends
eve_log, qubits_after_eve = eve_intercept_resend(qubits_sent)

# Eve -> Bob
bob_log = bob_measure_received_qubits(qubits_after_eve)

# Sifting (Alice/Bob same basis only)
keep_idx, alice_sifted, bob_sifted = sift_matching_bases(alice_log, bob_log)
M_max = len(alice_sifted)

if M_max == 0:
    print("No sifted bits available in this run.")
else:
    # True sifted QBER (for analysis only)
    full_errors = sum(
        1 for i in range(M_max)
        if alice_sifted[i][1] != bob_sifted[i][1]
    )
    qber_sifted = full_errors / M_max

    # Find minimum M that detects Eve in this FIRST simulated attempt
    min_M_detect = None
    detect_result = None

    for M in range(1, M_max + 1):
        # fixed seed per M -> deterministic first-attempt check
        res = compare_subset_and_extract_key(alice_sifted, bob_sifted, k=M, seed=100 + M)
        if res["errors"] > 0:
            min_M_detect = M
            detect_result = res
            break

    print(f"N = {N}")
    print(f"Sifted bits = {M_max}")
    print(f"Sifted QBER (analysis) = {qber_sifted:.3f}")

    if min_M_detect is None:
        print("No detection in checked subsets up to M_max (this run/seed).")
    else:
        print(f"Minimum M to detect Eve on first attempt (this run): {min_M_detect}")
        print(f"Errors found in test subset: {detect_result['errors']}")
        print(f"QBER on tested subset: {detect_result['qber_test']:.3f}")


N = 128
Sifted bits = 71
Sifted QBER (analysis) = 0.296
Minimum M to detect Eve on first attempt (this run): 4
Errors found in test subset: 1
QBER on tested subset: 0.250


## Question 14
Now, assume that Alice and Bob wanted to exchange an initial key of 128-bits long ($N = 128$); but, due to external reasons, they can only check 4 bits ($M = 4$). Could you find a situation in which the hacker’s interference was not detected? How many attempts did it take you to find such situation?

In [72]:
# Find a case where Eve is NOT detected with M=4
N = 128
M = 4
max_attempts = 10000

attempt = 0
found = False

while attempt < max_attempts:
    attempt += 1

    # Alice -> Eve
    alice_log, qubits_sent = alice_send_qubits_to_bob(N)

    # Eve intercepts/resends
    eve_log, qubits_after_eve = eve_intercept_resend(qubits_sent)

    # Eve -> Bob
    bob_log = bob_measure_received_qubits(qubits_after_eve)

    # Sift
    keep_idx, alice_sifted, bob_sifted = sift_matching_bases(alice_log, bob_log)
    M_max = len(alice_sifted)
    if M_max < M:
        continue

    # True errors in sifted key (analysis only)
    full_errors = sum(
        1 for i in range(M_max)
        if alice_sifted[i][1] != bob_sifted[i][1]
    )

    # Public check on only 4 bits
    res = compare_subset_and_extract_key(alice_sifted, bob_sifted, k=M)

    # "Not detected" = check passes, but Eve actually introduced errors
    if res["errors"] == 0 and full_errors > 0:
        found = True
        print(f"Undetected Eve interference found after {attempt} attempt(s).")
        print(f"Sifted bits: {M_max}")
        print(f"Actual sifted errors (hidden): {full_errors}")
        print(f"Checked bits M={M}, detected errors in check: {res['errors']}")
        break

if not found:
    print(f"No undetected case found in {max_attempts} attempts. Increase max_attempts.")



Undetected Eve interference found after 1 attempt(s).
Sifted bits: 65
Actual sifted errors (hidden): 17
Checked bits M=4, detected errors in check: 0
